# Dominion simulations

This is a workspace for running and testing my simulations

### Setup

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numpy import random
import sys
%matplotlib inline 

In [52]:
#automatically update dominionMarkov package
if 'dominionMarkov' in sys.modules:
    del sys.modules['dominionMarkov'] 
from dominionMarkov import *

### SQL database

The idea behind the database, is that every time I run a simulation, the results are saved.  That way, when I run the simulation a bunch of times, nothing is lost.

#Create database.  Should only be run once
init_sql_database()

In [54]:
#Print out the bottom row of each table in the database, for testing purposes
con = lite.connect('sim.db')
with con:
    cur = con.cursor()
    cur.execute('select * from sim_types where id = (select max(id) from sim_types)')
    print(cur.fetchone())
    cur.execute('select * from sim_results where id = (select max(id) from sim_results)')
    print(cur.fetchone())
con.close()

(4, 'Monte Herald Infinite', 'max_cards', 'stat_weight', 'fraction_heralds', None, None, 2)
(119, 2, 1, 38.0, 1000.0, 18.0, None, None, 5.149617364426216, 15.979, 0.0, 0)


#example code to delete rows
con = lite.connect('sim.db')
with con:
    cur = con.cursor()
    cur.execute('update sim_results set deleted = 1 where id = 47')
con.close()

### Monte Carlo simulations

A Monte Carlo simulation simulates many turns, determining the outcome of each by random number generation.  After many turns, statistics are compiled.  This method is the easiest to write, but also slow to execute and less precise.

First I try a Monte Carlo simulation of an infinite lab/copper deck.  It isn't really infinite, I limit it to 1000 cards drawn, after which point it is assumed to be infinite. The output is a tuple with (mean,stdev,reliability).  "reliability" is the probability of drawing infinite cards.  "mean" and "stdev" are statistics compiled from the finite turns.

In [15]:
for i in range(10):
    labsim = monte_lab_inf(i/10)
    print("L = ",i/10,": ",labsim.sim())

L =  0.0 :  (5.0, 0.0, 0.0)
L =  0.1 :  (5.645, 0.9428547077890682, 0.0)
L =  0.2 :  (6.721, 1.9373071516927811, 0.0)
L =  0.3 :  (8.563, 3.8018457359551014, 0.0)
L =  0.4 :  (14.776, 11.509032279040667, 0.0)
L =  0.5 :  (60.06734006734007, 86.93261422465457, 0.10899999999999999)
L =  0.6 :  (14.945736434108527, 12.188004063927611, 0.871)
L =  0.7 :  (8.833333333333334, 5.2094998693625945, 0.994)
L =  0.8 :  (10.0, 0.0, 0.999)
L =  0.9 :  (nan, nan, 1.0)


Next, we simulate a finite lab/copper deck.  Note that reliability is always zero here, since we always draw finite cards.

In [53]:
for i in range(0,20,2):
    labsim = monte_lab_fin(i,deck_size=i+20)
    print(i,"labs, 20 copper:",labsim.sim())

0 labs, 20 copper: (5.0, 0.0, 0.0)
2 labs, 20 copper: (5.499, 0.6511520559746438, 0.0)
4 labs, 20 copper: (6.086, 1.0385586165450647, 0.0)
6 labs, 20 copper: (6.903, 1.6098419177049677, 0.0)
8 labs, 20 copper: (7.736, 2.0655033284892084, 0.0)
10 labs, 20 copper: (8.877, 2.7371282395971126, 0.0)
12 labs, 20 copper: (10.053, 3.3967912800170668, 0.0)
14 labs, 20 copper: (12.319, 4.406953482849573, 0.0)
16 labs, 20 copper: (14.295, 5.266685390262077, 0.0)
18 labs, 20 copper: (15.979, 5.149617364426216, 0.0)


Next I have a simulation of an infinite village/smithy deck.

In [107]:
for i in range(4):
    for j in range(4):
        smithysim = monte_vsm_inf(i/8,j/8)
        print("V = ",i/8,", S = ", j/8, ": ",smithysim.sim())

V =  0.0 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.0 , S =  0.125 :  (5.597, 0.8559152995477953, 0.0)
V =  0.0 , S =  0.25 :  (5.481, 0.9785903126436519, 0.0)
V =  0.0 , S =  0.375 :  (4.85, 1.177921898938976, 0.0)
V =  0.125 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.125 , S =  0.125 :  (6.17, 1.8833746308156538, 0.0)
V =  0.125 , S =  0.25 :  (6.586, 2.9326786390602018, 0.0)
V =  0.125 , S =  0.375 :  (6.05, 3.4982138299423613, 0.0)
V =  0.25 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.25 , S =  0.125 :  (7.093, 3.9166760141732424, 0.0)
V =  0.25 , S =  0.25 :  (19.984848484848484, 43.175057950365826, 0.010000000000000009)
V =  0.25 , S =  0.375 :  (24.016233766233768, 47.628895024204986, 0.07599999999999996)
V =  0.375 , S =  0.0 :  (5.0, 0.0, 0.0)
V =  0.375 , S =  0.125 :  (9.502, 6.732606924512971, 0.0)
V =  0.375 , S =  0.25 :  (13.633136094674557, 18.493656000251544, 0.493)
V =  0.375 , S =  0.375 :  (4.699115044247788, 3.890958331693776, 0.774)


I can also simulate a more complicated deck, such as a [herald](http://wiki.dominionstrategy.com/index.php/File:Herald.jpg)/copper deck.

In [39]:
for i in range(10):
    heraldsim = monte_herald_inf(i/10)
    print("H = ",i/10,": ",heraldsim.sim())

H =  0.0 :  (5.0, 0.0, 0.0)
H =  0.1 :  (5.057, 0.2444401767304119, 0.0)
H =  0.2 :  (5.298, 0.6878924334516273, 0.0)
H =  0.3 :  (5.883, 1.4809831194176368, 0.0)
H =  0.4 :  (7.21, 3.1054629284536635, 0.0)
H =  0.5 :  (10.814, 7.66794653085166, 0.0)
H =  0.6 :  (42.20820512820513, 60.08483519038193, 0.025000000000000022)
H =  0.7 :  (15.614814814814816, 14.5240465666222, 0.865)
H =  0.8 :  (8.142857142857142, 2.948538205792901, 0.993)
H =  0.9 :  (nan, nan, 1.0)


### Markov Chain simulations

The Markov Chain method has a state vector, which each component of that vector indicating the probability that the game is in that particular state.  The state vector is transformed by matrix multiplication.  This method is harder to write, but is faster and more precise, since it does not require random number generation or compiling statistics.

In [52]:
#automatically update dominionMarkov package
if 'dominionMarkov' in sys.modules:
    del sys.modules['dominionMarkov'] 
from dominionMarkov import *